In [ ]:
#default_exp pretrained_models

# Pretrained Models

> A model zoo for to download and load pretrained models.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import ExceptionExpected, test_eq, test_close

In [ ]:
#export
import numpy as np
from fastai.vision.all import *

## Model Zoo

`MODELS` is a dictionary that maps model names to a url of the tarred model. `get_model` allows you to easily download a pretrained model (if it hasn't been downloaded already), from the `MODELS` dict. Then you can easily use it for inference by just passing in the path to an image file! :)

In [ ]:
#export
MODELS = {
    "resnet18_2021-04-08": "https://github.com/bwolfson97/plant_pathology/releases/download/v0.1.1-alpha/resnet18_2021-04-08.tar.gz"
}

Here's a list of the names of the pretrained models that you can pass to `get_model`.

In [ ]:
list(MODELS.keys())

['resnet18_2021-04-08']

In [ ]:
#export
def get_model(model_name: str):
    """Downloads and builds pretrained model.

    `model_name` must be in `MODELS` dict.
    """
    try:
        url = MODELS[model_name]
    except KeyError:
        raise KeyError(f"Invalid model name. Received: {model_name}")

    pickle_file = untar_data(url)
    return load_learner(pickle_file)

In [ ]:
#hide
# Test invalid model name
with ExceptionExpected(ex=KeyError): get_model("invalid_model_name")

## Prediction Wrapper 

`predict_leaf` is a simple wrapper over `Learner`'s `predict` that decodes the class probabilities for the user.

In [ ]:
#export 
@patch
def predict_leaf(self: Learner, image, decimals: int = 2):
    """Predict on image and return predicted class and decoded probabilities.

    Rounds probabilities to `decimals` decimal places.
    """
    predicted_class, _, probabilities = self.predict(image)

    # Round probabilities
    probabilities = np.around(probabilities.tolist(), decimals=decimals)

    # Decode probabilities using class names
    decoded_probabilities = dict(zip(self.dls.vocab, probabilities))

    # Format results
    return {
        "predicted_class": predicted_class,
        "probabilities": decoded_probabilities
    }

In [ ]:
#slow
model = get_model("resnet18_2021-04-08")
prediction = model.predict_leaf("../nbks/images/leaf.jpg")
prediction

{'predicted_class': 'rust',
 'probabilities': {'healthy': 0.05,
  'multiple_diseases': 0.04,
  'rust': 0.88,
  'scab': 0.03}}

In [ ]:
#slow
#hide
# Test prediction contains proper keys
test_eq(prediction.keys(), ["predicted_class", "probabilities"])

# Test probabilites add to ~1.0 (~ b/c they are rounded)
test_close(sum(prediction["probabilities"].values()), 1.0, eps=0.01)

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script("07_pretrained_models.ipynb")

Converted 07_pretrained_models.ipynb.
